In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import cv2
import os

In [ ]:
Y = pd.read_csv(r"2\img_align_celeba\list_attr_celeba.csv")
Y.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [ ]:
folder_path = r'2\img_align_celeba\img_align_celeba'

image_extensions = ('.jpg')

images = []
labels = []

for filename, y in zip(os.listdir(folder_path)[:25000], Y["Smiling"]):
    image_path = os.path.join(folder_path, filename)
    image = cv2.imread(image_path)

    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        images.append(image)
        labels.append(y)
    else:
        print(f"Warning: Failed to load {filename}")

In [ ]:
labels = [0 if x == -1 else x for x in labels]

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3, random_state=42, shuffle=True)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform_train = transforms.Compose([
    transforms.RandomCrop([178, 178]),
    transforms.RandomHorizontalFlip(),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
])
transform = transforms.Compose([
    transforms.CenterCrop([178, 178]),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
])

In [ ]:
class TensorImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.labels = labels
        self.transform = transform
        self.to_pil = ToPILImage()

        if self.transform:
            # Apply transform once to all images and store them
            self.images = torch.stack([
                self.transform(self.to_pil(img)) for img in images
            ])
        else:
            self.images = images

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = TensorImageDataset(X_train, y_train, transform=transform_train)
val_dataset = TensorImageDataset(X_val, y_val, transform=transform)

In [ ]:
torch.manual_seed(1)
mini_batch_size = 128
train_loader = DataLoader(train_dataset, mini_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, mini_batch_size, shuffle=False)

In [ ]:
model = nn.Sequential(
    # B, 3, 64, 64
    nn.Conv2d(
        in_channels=3, 
        out_channels=32,
        kernel_size=3, 
        padding=1
    ),
    # B, 32, 62, 62
    nn.ReLU(),
    # B, 32, 62, 62
    nn.MaxPool2d(kernel_size=2),
    # B, 32, 31, 31
    # nn.Dropout2d(p=0.5),
    # B, 32, 31, 31
    nn.Conv2d(
        in_channels=32, 
        out_channels=128,
        kernel_size=3, 
        padding=1
    ),
    nn.BatchNorm2d(128),
    # B, 128, 30, 30
    nn.ReLU(),
    # B, 128, 30, 30
    nn.MaxPool2d(kernel_size=2),
    # B, 128, 15, 15
    # nn.Dropout2d(p=0.5),
    # B, 128, 15, 15
    nn.Conv2d(
        in_channels=128,
        out_channels=256,
        kernel_size=3,
        padding=1
    ),
    nn.BatchNorm2d(256),
    # B, 256, 14, 14
    nn.ReLU(),
    # B, 256, 14, 14
    nn.MaxPool2d(kernel_size=2),
    # B, 256, 7, 7
    nn.Conv2d(
        in_channels=256,
        out_channels=256,
        kernel_size=3,
        padding=1
    ),
    nn.BatchNorm2d(256),
    # B, 256, 6, 6
    nn.ReLU(),
    # B, 256, 6, 6
    nn.AvgPool2d(kernel_size=8),
    # B, 256, 1, 1
    nn.Flatten(),
    # B, 256 * 1 * 1
    nn.Linear(256, 1)
)

model = model.to('cuda')

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
def train(model, num_epochs, train_dl, valid_dl):
    for epoch in range(num_epochs):
        model.train()
        
        losses = []
        
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.to('cuda').float()
            y_batch = y_batch.to('cuda').float()
            pred = model(x_batch).squeeze(1)

            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad()
            
            with torch.no_grad():
                li = loss.item()
                # print(li)
                losses.append(li)
                
        print(f'Epoch {epoch+1}, Loss: {np.mean(losses):.4f}')
        if (epoch + 1) % 5 == 0:
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for x_batch, y_batch in valid_dl:
                    x_batch = x_batch.to('cuda').float()
                    y_batch = y_batch.to('cuda').float()
                    pred = model(x_batch).squeeze(1)
                    predicted = (pred > 0.5).float()
                    total += y_batch.size(0)
                    correct += (predicted == y_batch).sum().item()

                accuracy = 100 * correct / total
                print(f'Epoch {epoch+1}, Validation Accuracy: {accuracy:.2f}%')

torch.manual_seed(1)
num_epochs = 30
train(model, num_epochs, train_loader, val_loader)

Epoch 1, Loss: 0.0021
Epoch 2, Loss: 0.0021
Epoch 3, Loss: 0.0018
Epoch 4, Loss: 0.0024
Epoch 5, Loss: 0.0019
Epoch 5, Validation Accuracy: 85.45%
Epoch 6, Loss: 0.0018
Epoch 7, Loss: 0.0015
Epoch 8, Loss: 0.0013
Epoch 9, Loss: 0.0016
Epoch 10, Loss: 0.0014
Epoch 10, Validation Accuracy: 84.53%
Epoch 11, Loss: 0.0012
Epoch 12, Loss: 0.0013
Epoch 13, Loss: 0.0011
Epoch 14, Loss: 0.0011
Epoch 15, Loss: 0.0010
Epoch 15, Validation Accuracy: 86.74%
Epoch 16, Loss: 0.0008
Epoch 17, Loss: 0.0010
Epoch 18, Loss: 0.0013
Epoch 19, Loss: 0.0008
Epoch 20, Loss: 0.0009
Epoch 20, Validation Accuracy: 86.56%
Epoch 21, Loss: 0.0009
Epoch 22, Loss: 0.0008
Epoch 23, Loss: 0.0008
Epoch 24, Loss: 0.0008
Epoch 25, Loss: 0.0007
Epoch 25, Validation Accuracy: 86.37%
Epoch 26, Loss: 0.0006
Epoch 27, Loss: 0.0006
Epoch 28, Loss: 0.0007
Epoch 29, Loss: 0.0005
Epoch 30, Loss: 0.0005
Epoch 30, Validation Accuracy: 86.00%
